In [251]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [252]:
# ZENTRALE PARAMETER (einfach hier ändern)

ARZT_ID = 1
PFLEGEKRAFT_ID = 1
KRANKENHAUS_ID = 1
TODAY = '2025-12-12'
PATIENT_VORNAME = "Rayan"
PATIENT_NACHNAME = "Jost"


def execute_print_query(cursor,query):
    cursor.execute(query)
    column_names = [desc[0] for desc in cur.description]
    print(column_names)
    for row in cur.fetchall():
        print(row)


Dr. Luc Thommen kommt zur Arbeit und möchte wissen, welche Patienten er heute betreut und wo sie sich befinden.

In [253]:
query = f"""
-- Zeigt alle heutigen Patienten eines Arztes inklusive Station und Event
SELECT
    p.patient_id,
    p.vorname || ' ' || p.nachname AS patient_name,
    s.name AS station,
    e.datum,
    e.beschreibung AS event_beschreibung
FROM arzt a
JOIN arzt_event ae ON a.arzt_id = ae.arzt_id
JOIN event e ON ae.event_id = e.event_id
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
JOIN station s ON p.station_id = s.station_id
WHERE a.arzt_id = {ARZT_ID}
  AND e.datum = DATE '{TODAY}'
ORDER BY s.name, patient_name;
"""
execute_print_query(cur, query)


['patient_id', 'patient_name', 'station', 'datum', 'event_beschreibung']
(4, 'Dilara Leu', 'Chirurgie', datetime.date(2025, 12, 12), 'Medikamentengabe')
(120, 'Hulda Bättig', 'Chirurgie', datetime.date(2025, 12, 12), 'Nachkontrolle')
(28, 'Kelly Gerber', 'Chirurgie', datetime.date(2025, 12, 12), 'Visite')
(23, 'Rayan Jost', 'Chirurgie', datetime.date(2025, 12, 12), 'Medikamentengabe')
(133, 'Valentin Bernasconi', 'Chirurgie', datetime.date(2025, 12, 12), 'Medikamentengabe')


Der Arzt erkennt, dass Tanja Bernasconi heute mehrfach auf seiner Liste auftaucht.
Bevor er das Zimmer betritt, möchte er sich einen Überblick verschaffen, welche Behandlungen sie bisher hatte und wann diese stattgefunden haben.

In [254]:
query = f"""
-- Zeigt alle bisherigen Behandlungen einer bestimmten Patientin (Patientendossier)
SELECT
    b.behandlungs_id,
    p.vorname || ' ' || p.nachname AS patient_name,
    b.beschreibung,
    b.abgeschlossen,
    b.start_datum,
    b.end_datum
FROM patient p
JOIN behandlung b ON p.patient_id = b.patient_id
WHERE p.vorname = '{PATIENT_VORNAME}'
  AND p.nachname = '{PATIENT_NACHNAME}'
ORDER BY b.start_datum;
"""
execute_print_query(cur, query)


['behandlungs_id', 'patient_name', 'beschreibung', 'abgeschlossen', 'start_datum', 'end_datum']
(3, 'Rayan Jost', 'Atemwegsinfekt Behandlung', True, datetime.date(2025, 11, 13), datetime.date(2025, 12, 12))


Der Arzt hat sich einen Überblick über Tanja Bernasconi Krankengeschichte verschafft.
Jetzt möchte er wissen, welche Medikamente sie im Verlauf ihrer Behandlungen erhalten hat, um Wechselwirkungen zu vermeiden und die heutige Therapie richtig zu planen.

In [255]:
query = f"""
-- Zeigt alle Medikamente, die einer Patientin im Rahmen ihrer Behandlungen verabreicht wurden
SELECT DISTINCT
    m.medikament_id,
    m.name AS medikament_name,
    m.wirkstoff,
    e.datum AS event_datum,
    e.beschreibung AS event_typ
FROM patient p
JOIN behandlung b ON p.patient_id = b.patient_id
JOIN event e ON b.behandlungs_id = e.behandlung_id
JOIN event_medikament em ON e.event_id = em.event_id
JOIN medikament m ON em.medikament_id = m.medikament_id
WHERE p.vorname = '{PATIENT_VORNAME}'
  AND p.nachname = '{PATIENT_NACHNAME}'
ORDER BY e.datum, m.name;
"""
execute_print_query(cur, query)


['medikament_id', 'medikament_name', 'wirkstoff', 'event_datum', 'event_typ']
(4, 'Amoxicillin', 'Amoxicillin', datetime.date(2025, 12, 12), 'Medikamentengabe')
(1, 'Paracetamol', 'Paracetamol', datetime.date(2025, 12, 12), 'Medikamentengabe')


Nachdem der Arzt weiß, welche Behandlungen und Medikamente Tanja Bernasconi betreffen,
möchte er jetzt ganz konkret wissen, wo sie sich aktuell befindet, um persönlich bei ihr vorbeizugehen.

In [256]:
query = """
SELECT
    p.patient_id,
    p.vorname || ' ' || p.nachname AS patient_name,
    s.name AS station,
    r.art AS raum_art,
    r.raumnummer,
    b.bett_id
FROM patient p
JOIN bett b ON b.patient_id = p.patient_id
JOIN raum r ON b.raum_id = r.raumnummer
JOIN station s ON r.station_id = s.station_id
WHERE p.vorname = %s
  AND p.nachname = %s
  AND b.status = 'belegt';
"""

execute_print_query(
    cur,
    cur.mogrify(query, (PATIENT_VORNAME, PATIENT_NACHNAME)).decode()
)

['patient_id', 'patient_name', 'station', 'raum_art', 'raumnummer', 'bett_id']
(23, 'Rayan Jost', 'Chirurgie', 'Patientenzimmer', 8, 6)


Nachdem der Arzt weiss, wo sich Tanja Bernasconi befindet, möchte er sich kurz abstimmen.
Bevor er den Rundgang startet, schaut er nach, welche Pflegekräfte heute auf seiner Station eingeteilt sind, um Übergaben und Absprachen effizient zu machen.

In [257]:
query = """
SELECT
    s.name AS station,
    pk.pflege_id,
    pk.vorname || ' ' || pk.nachname AS pflegekraft_name,
    pk.telefonnummer
FROM arzt a
JOIN station s ON a.station_id = s.station_id
JOIN pflegekraft pk ON pk.station_id = s.station_id
WHERE a.arzt_id = %s
ORDER BY pflegekraft_name;
"""

execute_print_query(
    cur,
    cur.mogrify(query, (ARZT_ID,)).decode()
)


['station', 'pflege_id', 'pflegekraft_name', 'telefonnummer']
('Chirurgie', 1, 'Andrin Lüthi', '+41 79 002001')
('Chirurgie', 4, 'Egzon Merz', '+41 79 002004')
('Chirurgie', 3, 'Gerold Fuchs', '+41 79 002003')
('Chirurgie', 2, 'Jeffrey Kälin', '+41 79 002002')
